In [48]:
%pip install gurobipy
from gurobipy import Model, GRB, quicksum
import pandas as pd
import numpy as np

# Example Data
pilots = ['Pilot1', 'Pilot2', 'Pilot3', 'Pilot4', 'Pilot5', 'Pilot6', 'Pilot7']  # List of pilots
time_periods = ['T1', 'T2', 'T3', 'T4']  # Time periods
qualifications = {
    'Pilot1': ['T3'],
    'Pilot2': ['T3','T4'],
    'Pilot3': ['T1','T2'],
    'Pilot4': ['T3'],
    'Pilot5': ['T2'],
    'Pilot6': ['T1', 'T2'],
    'Pilot7': ['T1']
}
costs = np.array([100, 150, 90, 120, 180, 100, 160])  # Cost of assigning pilots to duties
fixed_costs = np.array([50, 75, 25, 60, 40, 30, 45])  # Fixed costs per pilot
penalty_cost = 40
nb_pilots = len(pilots)
nb_time = len(time_periods)

# Maximum allowable flying hours for each pilot
max_flying_hours = np.array([130,125,160,140,150,130,130])

# Maximum block hours for each pilot
max_block_hours = np.array([40,40,50,50,55,55,60])

# Total block hours required at each time period
required_block_hours = np.array([200,250,300,280])

# Define the model
m = Model('Pilot_Scheduling_Optimization')


# Decision variables
x = m.addVars(nb_pilots, nb_time, vtype=GRB.BINARY, name='assignment')
y = m.addVars(nb_pilots, vtype=GRB.BINARY, name='fixed_cost_incurred')
b = m.addVars(nb_pilots, nb_time, vtype=GRB.CONTINUOUS, name='block_hours')
shortage = m.addVars(nb_time, vtype=GRB.CONTINUOUS, name='shortage_decision') 
    
# Objective function
m.setObjective(
    quicksum(costs[p] * x[p, t] + fixed_costs[p] * y[p] for p in range(nb_pilots) for t in range(nb_time)) +
   quicksum(penalty_cost * shortage[t] for t in range(nb_time)),
    GRB.MINIMIZE
)

# Constraints
# Availability and Qualification Constraint
m.addConstrs((x[p, t] == 0) for p in range(nb_pilots) for t in range(nb_time) if t in qualifications.get(p, []))

# Ensure coverage (Example assumes at least 1 pilot needed per time period)
m.addConstrs((quicksum(x[p, t] for p in range(nb_pilots)) >= 1) for t in range(nb_time))

# Ensure that each pilot's total assigned block hours do not exceed their maximum allowable flying hours
m.addConstrs((quicksum(b[p, t] for t in range(nb_time)) <= max_flying_hours[p]) for p in range(nb_pilots))

# The block hours assigned to a pilot for a duty must be less than or equal to the maximum block hours they can fly
m.addConstrs((b[p, t] <= max_block_hours[p] * x[p, t]) for p in range(nb_pilots) for t in range(nb_time))

# Coverage constraint with block hours
m.addConstrs((quicksum(b[p, t] for p in range(nb_pilots)) + shortage[t] == required_block_hours[t]) for t in range(nb_time))

# constraint for the shortages each time period
for t in range(nb_time):
    m.addConstr(shortage[t] <= 20, name=f'shortage_limit_{t}')


# Update the model and solve
m.update()
m.optimize()

# Optimal objective value
print(f'The optimal objective value is {m.ObjVal:g}')
# Output results

# Block hours assigned to each pilot at each time period
block_hours_df = pd.DataFrame(
    [[b[p, t].X for t in range(nb_time)] for p in range(nb_pilots)],
    index=pilots,
    columns=[f"Block Hours {t+1}" for t in range(nb_time)]
)
print("\nBlock Hours Assigned:")
print(block_hours_df)

# Print duty assignment
assignment_df = pd.DataFrame(
    [[x[p, t].X for p in range(nb_pilots)] for t in range(nb_time)],
    index=time_periods,
    columns=[f"Pilot {p+1}" for p in range(nb_pilots)]
)
print("\nDuty Assignment Matrix:")
print(assignment_df)

# Print shortage for each time period
shortage_values = [shortage[t].X for t in range(nb_time)]
print("\nShortage for Each Time Period:")
for t, value in enumerate(shortage_values):
    print(f"Time {t+1}: {value:g}")

Note: you may need to restart the kernel to use updated packages.
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: 13th Gen Intel(R) Core(TM) i5-1335U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 47 rows, 67 columns and 148 nonzeros
Model fingerprint: 0x147a35f4
Variable types: 32 continuous, 35 integer (35 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+01]
  Objective range  [4e+01, 3e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective 6340.0000000
Presolve removed 4 rows and 7 columns
Presolve time: 0.00s
Presolved: 43 rows, 60 columns, 147 nonzeros
Variable types: 32 continuous, 28 integer (28 binary)

Root relaxation: objective 5.091689e+03, 61 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj

In [45]:
# Print duty assignment
assignment_df = pd.DataFrame(
    [[x[p, t].X for p in range(nb_pilots)] for t in range(nb_time)],
    index=time_periods,
    columns=[f"Pilot {p+1}" for p in range(nb_pilots)]
)
print("\nDuty Assignment Matrix:")
print(assignment_df)


Duty Assignment Matrix:
    Pilot 1  Pilot 2  Pilot 3  Pilot 4  Pilot 5  Pilot 6  Pilot 7
T1      1.0      1.0      1.0      1.0      1.0      1.0     -0.0
T2      1.0      1.0      1.0     -0.0      1.0     -0.0      1.0
T3      1.0      1.0      1.0      1.0      1.0      1.0      1.0
T4      1.0      1.0      1.0      1.0     -0.0      1.0      1.0


In [46]:
# Print shortage for each time period
shortage_values = [shortage[t].X for t in range(nb_time)]
print("\nShortage for Each Time Period:")
for t, value in enumerate(shortage_values):
    print(f"Time {t+1}: {value:g}")


Shortage for Each Time Period:
Time 1: 20
Time 2: 5
Time 3: 20
Time 4: 20
